In [ ]:
!pip install bert-tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 1.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DataPath = "/content/drive/MyDrive/Magang/ThreatDetection/MultiLabeld/"

In [ ]:
# "ThreathDataset.csv".encode('utf-8')

In [ ]:
# df_threath = pd.read_csv("ThreathDataset.csv", encoding='latin-1')
df_threath = pd.read_csv(f"{DataPath}dataset.csv", encoding='latin-1')

In [ ]:
df_threath.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


In [ ]:
# df_threath.drop(columns = ["HS_Other"], inplace = True)

In [ ]:
def replace_slang_with_formal(word, dfslang, columns_slang:str, columns_formal:str):
    match = dfslang[dfslang[columns_slang].str.lower() == word.lower()]
    if not match.empty:
        return match[columns_formal].values[0]
    return word

def replace_slang_in_corpus(corpus, dfslang, columns_slang:str, columns_formal:str):
    updated_corpus = []
    for sentence in corpus:
        updated_sentence = ' '.join([replace_slang_with_formal(word, dfslang=dfslang, columns_slang=columns_slang, columns_formal= columns_formal) for word in sentence.split()])
        updated_corpus.append(updated_sentence)
    return updated_corpus
# dfslang2.columns = ['slang', 'formal']
df_kamusAlay = pd.read_csv(f"{DataPath}kamusalay.csv", encoding='latin-1', header= None)
df_kamusAlay.rename(columns={0: 'alay', 1: 'formal'}, inplace = True)
formal_tweet_list = replace_slang_in_corpus(df_threath["Tweet"], df_kamusAlay, 'alay', 'formal')
df_formal = pd.DataFrame(formal_tweet_list)

In [ ]:
df_formal.to_csv("formal.csv")

In [ ]:
from google.colab import files
files.download('formal.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df = pd.read_csv("formal.csv", encoding='latin-1', header= None)

In [ ]:
df.dropna(inplace = True)

In [ ]:
df_threath.iloc[:, 1:]

,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Weak,HS_Moderate,HS_Strong
0,1,1,1,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0
4,1,1,0,1,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
13164,1,1,1,0,0,0,1,0,1,0,0
13165,0,1,0,0,0,0,0,0,0,0,0
13166,0,0,0,0,0,0,0,0,0,0,0
13167,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.rename(columns= {1 : "Tweet"}, inplace = True)

In [ ]:
df.drop(columns = 0, inplace= True)

In [ ]:
df.reset_index(inplace= True)

In [ ]:
df.drop(columns = 'index', inplace = True)

In [ ]:
df = pd.concat([df, df_threath.iloc[:, 1:]], axis = 1)

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
labels = df.iloc[:, 1:].columns.to_list()

In [ ]:
labels

['HS',
 'Abusive',
 'HS_Individual',
 'HS_Group',
 'HS_Religion',
 'HS_Race',
 'HS_Physical',
 'HS_Gender',
 'HS_Weak',
 'HS_Moderate',
 'HS_Strong']

In [ ]:
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}

In [ ]:
def prerocessing_indo(sentences):
    # case Folding
    sentences = [sentence.lower() for sentence in sentences]

    # tokenizer
    tokenizer     = RegexpTokenizer(r'\w+')
    # hasil_token   = tokenizer.tokenize(sentences[1])
    token_sentence = [tokenizer.tokenize(sentence) for sentence in sentences]

    # stopword
    wordlist= set(stopwords.words('indonesian')+["all", "halaman", "off", "mengunggah file"])
    sw_sentences = [[word for word in sentence if word not in wordlist]for sentence in token_sentence]
    cleaned_sentences = [' '.join(sentence) for sentence in sw_sentences]

    sentence = cleaned_sentences
    return sentence

In [ ]:
# from transformers import BertTokenizer, TFBertModel, Trainer, TrainingArguments, TFBertForSequenceClassification,  TFBertTokenizer
# model_name='cahya/bert-base-indonesian-522M'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=10, id2label = id2label, label2id = label2id)

In [ ]:
df["Tweet"] = prerocessing_indo(df["Tweet"].to_list())

In [ ]:
df.Tweet

0        cowok berusaha melacak perhatian gue lantas re...
1        rt user pengguna telat tau elu edan sarap gue ...
2        41 kadang berfikir percaya tuhan jatuh berkali...
3                 pengguna pengguna n nku tau matamu sipit
4        pengguna pengguna kaum cebong kafir dongoknya ...
                               ...                        
13164       pengguna berbicara ndasmu congor sekata anjyng
13165                           pengguna kasur enak kunyuk
13166    pengguna hati hati bisu g n nlagi bosan duh xf...
13167    pengguna pengguna pengguna pengguna bom real m...
13168              pengguna situ ngasih foto ya kutil onta
Name: Tweet, Length: 13169, dtype: object

In [ ]:
# model_name = "indolem/indobertweet-base-uncased"
model_name='cahya/bert-base-indonesian-522M'

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 50.9 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, TFBertModel, Trainer, TrainingArguments, TFBertForSequenceClassification,  TFBertTokenizer


In [ ]:
tokenizer = BertTokenizer.from_pretrained(model_name)

In [ ]:
model = TFBertForSequenceClassification.from_pretrained(model_name,
                                                        problem_type="multi_label_classification",
                                                        num_labels=len(labels),
                                                        id2label=id2label,
                                                        label2id=label2id)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 0].values, df.iloc[:, 1:].values, test_size=0.3)

In [ ]:
df.iloc[:, 1:].values

array([[1, 1, 1, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 0, 0]])

In [ ]:
encode_input = tokenizer(list(X_train), padding= True, truncation=True, max_length=512, return_tensors="pt" )
encode_input

{'input_ids': tensor([[    3,  4196, 12984,  ...,     2,     2,     2],
        [    3, 12513,  2664,  ...,     2,     2,     2],
        [    3,  4196,  4196,  ...,     2,     2,     2],
        ...,
        [    3,  4196,  4196,  ...,     2,     2,     2],
        [    3,  8488, 26075,  ...,     2,     2,     2],
        [    3,  4659, 15071,  ...,     2,     2,     2]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
import tensorflow as tf

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((dict(encode_input), y_train)).batch(2)

In [ ]:
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def compute_loss(labels, predictions):
    per_label_loss = loss_object(labels, predictions)
    return tf.reduce_mean(per_label_loss)

In [ ]:
# Metrik untuk melacak akurasi
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

In [ ]:
# Siklus pelatihan
epochs = 3
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
for epoch in range(epochs):
    print("\nStart epoch", epoch + 1)
    for step, (inputs, labels) in enumerate(dataset):
        with tf.GradientTape() as tape:
            # Mendapatkan output dari model
            outputs = model(inputs)
            logits = outputs.logits
            probs = tf.sigmoid(logits)
            # Hitung loss
            loss = compute_loss(labels, logits)

        # Hitung gradien dan lakukan backpropagation
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        if step % 50 == 0:
            print("Step:", step, "Loss:", loss.numpy())

    # Evaluasi F1-score pada akhir setiap epoch
    predictions = model.predict(dataset)
    predictions = tf.sigmoid(predictions.logits).numpy()

    # Konversi predictions menjadi biner (0 atau 1) berdasarkan threshold 0.5
    binary_predictions = (predictions > 0.5).astype(int)

    labels_np = tf.convert_to_tensor(labels).numpy()
    # f1 = f1_score(labels_np, binary_predictions, average='micro')
    # print("F1-score after epoch", epoch + 1, ":", f1)


Start epoch 1
Step: 0 Loss: 0.025720967
Step: 50 Loss: 0.7992046
Step: 100 Loss: 0.023573808
Step: 150 Loss: 0.747875
Step: 200 Loss: 0.14554603
Step: 250 Loss: 0.076599285
Step: 300 Loss: 0.060638204
Step: 350 Loss: 0.056616798
Step: 400 Loss: 0.08305539
Step: 450 Loss: 0.031440485
Step: 500 Loss: 0.25271147
Step: 550 Loss: 0.13625138
Step: 600 Loss: 0.17278522
Step: 650 Loss: 0.115448564
Step: 700 Loss: 0.3230149
Step: 750 Loss: 0.12766705
Step: 800 Loss: 0.108439475
Step: 850 Loss: 0.1920366
Step: 900 Loss: 0.25115535
Step: 950 Loss: 0.055237804
Step: 1000 Loss: 0.21453819
Step: 1050 Loss: 0.09161055
Step: 1100 Loss: 0.07784565
Step: 1150 Loss: 0.030260488
Step: 1200 Loss: 0.078751445
Step: 1250 Loss: 0.17022844
Step: 1300 Loss: 0.49017143
Step: 1350 Loss: 0.06588384
Step: 1400 Loss: 0.12346402
Step: 1450 Loss: 0.33957556
Step: 1500 Loss: 0.13713074
Step: 1550 Loss: 0.26229614
Step: 1600 Loss: 0.08599584
Step: 1650 Loss: 0.2707282
Step: 1700 Loss: 0.0171108
Step: 1750 Loss: 0.34177

ValueError: ignored

In [ ]:
!mkdir model
!mkdir token
# Simpan model ke direktori tertentu
model.save_pretrained("model")

# Simpan tokenizer ke direktori yang sama (opsional, tapi disarankan)
tokenizer.save_pretrained("token")

('token/tokenizer_config.json',
 'token/special_tokens_map.json',
 'token/vocab.txt',
 'token/added_tokens.json')

In [ ]:
!zip -r model.zip model
!zip -r token.zip token

  adding: model/ (stored 0%)
  adding: model/tf_model.h5 (deflated 7%)
  adding: model/config.json (deflated 54%)
  adding: token/ (stored 0%)
  adding: token/vocab.txt (deflated 51%)
  adding: token/special_tokens_map.json (deflated 42%)
  adding: token/tokenizer_config.json (deflated 47%)


In [ ]:
from google.colab import files
files.download('model.zip')
files.download('token.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
y_test

array([[1, 1, 0, ..., 0, 1, 0],
       [1, 0, 1, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
X_test

array(['urusan mantan habis habis diomongin anehnya mantan mantan gue pas nikah nlp gue kayak orang doa restu seminggu nikah mengajak kemping geblek',
       'rt pengguna pengguna jgb mimpi kroco kroco inginnya impor stok',
       'pengguna banci merokok', ..., 'pengguna pengguna buddha teroris',
       'pilihan kepala daerah dki dinyatakan ahok elektabilitasnya 80 calon tunggal mengalahkan ahok cukong lupa media sosmed dimonopoli kebenaran nya pengguna maju calon gubernur dki1 elektabilitas ahok tergerus',
       'ya workshop negosiasi orang china'], dtype=object)

In [ ]:
!unzip model.zip
!unzip token_threath.zip


Archive:  model.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of model.zip or
        model.zip.zip, and cannot find model.zip.ZIP, period.
Archive:  token_threath.zip
   creating: token/
  inflating: token/vocab.txt         
  inflating: token/special_tokens_map.json  
  inflating: token/tokenizer_config.json  
